# Part 2 Data Wrangling
---
## Papers Past Topic Modeling

<br/>

Ben Faulks - bmf43@uclive.ac.nz

Xiandong Cai - xca24@uclive.ac.nz

Yujie Cui - ycu23@uclive.ac.nz

In [1]:
import sys, subprocess
sys.path.insert(0, '../utils') # for import customed modules
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *
from utils_data import conf_pyspark, load_dataset

# intiate PySpark
sc, spark = conf_pyspark()

sc

[('spark.app.name', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.port', '38900'),
 ('spark.driver.memory', '62g'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '192.168.1.207'),
 ('spark.app.id', 'local-1547627998888'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '6'),
 ('spark.driver.maxResultSize', '4g')]


<SparkContext master=local[*] appName=local>

## 1 Load Data

**Load raw dataset:**

In [2]:
df = load_dataset('papers_past', spark)

nrow_raw = df.count()
print('Shape of dataframe: ({}, {})'.format(nrow_raw, len(df.columns)))
df.sample(False, 0.00001).limit(10).show()

Shape of dataframe: (16731578, 6)
+--------+--------------------+------------------+--------------------+--------------------+--------------------+
|      id|                 url|         publisher|                time|               title|             content|
+--------+--------------------+------------------+--------------------+--------------------+--------------------+
|16657613|http://api.digita...|Poverty Bay Herald|1917-02-26T00:00:...|KILLED IN MINE SH...|KILLED IN MINE SH...|
|16707091|http://api.digita...|Poverty Bay Herald|1918-03-09T00:00:...|MUTINY IN POLISH ...|MUTINY IN POLISH ...|
|16810288|http://api.digita...|Poverty Bay Herald|1920-02-06T00:00:...|LATE NEW ZEALAND ...|LATE NEW ZEALAND ...|
| 7987898|http://api.digita...|Poverty Bay Herald|1891-10-19T00:00:...|Page 2 Advertisem...|rummer Fashions. ...|
| 8192651|http://api.digita...|Poverty Bay Herald|1891-07-21T00:00:...|PARIS, July 20. (...|PARIS, July 20.Ou...|
| 1870085|http://api.digita...|      Bruce Herald|1866

## 2 Missing Values

**Check empty values:**

In [3]:
# Count null
print('Print Null:')
df.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ''), c)).alias(c) for c in df.columns]).show()

Print Null:
+---+---+---------+----+-----+-------+
| id|url|publisher|time|title|content|
+---+---+---------+----+-----+-------+
|  0|  0|        0|   0|    0|  56232|
+---+---+---------+----+-----+-------+



**Drop rows with empty document:**

In [4]:
# Clean NA to avoid nonetype.
df = df.na.drop(subset=['content'])

**Check again:**

In [5]:
# Count null
print('Print Null:')
df.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ''), c)).alias(c) for c in df.columns]).show()

Print Null:
+---+---+---------+----+-----+-------+
| id|url|publisher|time|title|content|
+---+---+---------+----+-----+-------+
|  0|  0|        0|   0|    0|      0|
+---+---+---------+----+-----+-------+



## 3 Duplicate Values

**The "id" should be unique, check duplication:**

In [6]:
print('Duplicated id number: ', df.count() - df.select('id').distinct().count())

Duplicated id number:  543700


**There are duplicated "id" in the dataset, show three of them:**

In [7]:
df.groupBy('id').count().where(F.col('count')>1).show(3)

+--------+-----+
|      id|count|
+--------+-----+
|10036037|    2|
|10059447|    2|
|10099968|    2|
+--------+-----+
only showing top 3 rows



**Select the first one to check detail:**

In [8]:
df.filter(df.id == 10036037).show()

+--------+--------------------+------------------+--------------------+--------------------+--------------------+
|      id|                 url|         publisher|                time|               title|             content|
+--------+--------------------+------------------+--------------------+--------------------+--------------------+
|10036037|http://api.digita...|Poverty Bay Herald|1898-01-06T00:00:...|THE EASTERN SITUA...|THE EASTERN SITUA...|
|10036037|http://api.digita...|Poverty Bay Herald|1898-01-06T00:00:...|THE EASTERN SITUA...|THE EASTERN SITUA...|
+--------+--------------------+------------------+--------------------+--------------------+--------------------+



**Check difference of the content:**

In [9]:
import difflib

str1 = df.filter(df.id == 10036037).select('content').collect()[0]['content']
#print(str1 + '\n')

str2 = df.filter(df.id == 10036037).select('content').collect()[1]['content']
#print(str2 + '\n')

diff = difflib.SequenceMatcher(None, str1, str2).ratio()

print('Similarity: ', diff)

Similarity:  0.9994846688997681


**The two duplicates are very close, drop one of them:**

In [10]:
df = df.drop_duplicates(subset=['id'])

**Check duplicate again:**

In [11]:
print('Duplicated id number: ', df.count() - df.select('id').distinct().count())

Duplicated id number:  0


## 4 Abnormal Values

**There are should have 68 publishers, check numbers:**

In [12]:
n = df.select('publisher').distinct().count()
print(n)
if n == 68:
    print('Correct! no abnormal values in publishers.')
else:
    print('Error! abnormal values in publishers.')

68
Correct! no abnormal values in publishers.


## 5 Extract Features

### 5.1 Date

**For history documents, it only need date as time unit, we extract "date" column from "time" column:**

In [13]:
# extract feature date
df = df.withColumn('date', df['time'].cast(DateType()))

**Check schema of the dataframe:**

In [14]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- time: string (nullable = true)
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)
 |-- date: date (nullable = true)



**Check date range has abnormal values:**

In [15]:
start, end = df.select(F.min('date'), F.max('date')).first()
start, end

(datetime.date(1839, 8, 21), datetime.date(1945, 12, 31))

### 5.2 Advertisements

**Check "title" column to see if it is possible to extract features:**

In [16]:
df.sample(False, 0.00001).limit(10).select('id', 'title').show(10, False)

+--------+------------------------------------------------------------------+
|id      |title                                                             |
+--------+------------------------------------------------------------------+
|24123878|Page 4 Advertisements Column 4 (Otago Daily Times, 19 August 1898)|
|10804176|PARLIAMENTARY. (Grey River Argus, 09 October 1896)                |
|19015395|Page 12 Advertisements Column 5 (Evening Post, 04 August 1925)    |
|23545190|SPORTING. (Feilding Star, 27 May 1910)                            |
|6489339 |Page 1 Advertisements Column 1 (Poverty Bay Herald, 27 May 1886)  |
|23991783|THE WEATHER AND THE CROPS. (Otago Daily Times, 08 February 1892)  |
|28298850|Turc Bell Skirt. (Auckland Star, 30 September 1893)               |
|28135530|ANTI-CHINESE LEGISLATION. (Auckland Star, 22 March 1888)          |
|10994388|PROSPECTING NEAR GISBORNE. (Thames Star, 15 October 1895)         |
|17111566|HIS MAJESTY'S THEATRE. (Evening Post, 17 May 1918)    

**The "title" column specified advertisement, we extract "ads" column from "title" column:**

In [17]:
# extract feature ads
df = df.withColumn('ads', df.title.contains('dvertisement'))

### 5.3 Title

**The title consists of three parts: "real title" ("publisher", "date"), we only need "real title" part. Extract real title:**

In [18]:
# remove redandunt parts of title
df = df.withColumn('title_', F.regexp_extract(F.col('title'), '(.*)(\s\(.*\))', 1))

**Check if some titles are not the form "title ("publisher", "date"), which will lead to "title_" column is empty string:**

In [19]:
df.where(F.col('title_') == '').select(['id', 'title_', 'title']).show(5, False)

+--------+------+---------------------+
|id      |title_|title                |
+--------+------+---------------------+
|3656781 |      |Untitled Illustration|
|4832017 |      |Untitled Illustration|
|5417742 |      |Untitled Illustration|
|12676570|      |Untitled Illustration|
|12777321|      |Untitled Illustration|
+--------+------+---------------------+
only showing top 5 rows



**Change empty string in "title_" column to "Untitled Illustration":**

In [20]:
df = df.withColumn(
    'title_',
    F.when(
        F.col('title_') == '',
        F.lit('Untitled Illustration')
    ).otherwise(
        F.col('title_')
    )
)

**Check empty string again:**

In [21]:
# Count null
print('Print Null:')
df.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ''), c)).alias(c) for c in df.columns]).show()

Print Null:
+---+---+---------+----+-----+-------+----+---+------+
| id|url|publisher|time|title|content|date|ads|title_|
+---+---+---------+----+-----+-------+----+---+------+
|  0|  0|        0|   0|    0|      0|   0|  0|     0|
+---+---+---------+----+-----+-------+----+---+------+



### 5.4 Region

**In the website of [Papers Past](https://paperspast.natlib.govt.nz), we could find the publisher-region relationship in the [Explore all newspapers](https://paperspast.natlib.govt.nz/newspapers/all#region) webpage. Based on this webpage, we could extract region feature from "publisher" column. Here we saved [the webpage](https://paperspast.natlib.govt.nz/newspapers/all#region) and crawling the publisher-region relationship into a dataframe for extract feature.**

In [22]:
from bs4 import BeautifulSoup

# read webpage
path = r'../temp/Papers Past _ Explore all newspapers.html'
with open(path, 'r') as f:
    html = f.read()

# get table 
soup = BeautifulSoup(html, "html.parser")
table = soup.find('table', attrs={'class':'table datatable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

# transform table to pandas dataframe
df_region = pd.DataFrame(res, columns=['publisher_', 'region', 'start_', 'end_']) # column_ means it will be drop later

# transform pandas dataframe to pyspark dataframe
df_region = spark.createDataFrame(df_region).orderBy('region')

In [23]:
print('Shape of dataframe: ({}, {})'.format(df_region.count(), len(df_region.columns)))
df_region.sample(False, 0.1).show(16, False)

Shape of dataframe: (148, 4)
+--------------------------------------+-----------------+------+----+
|publisher_                            |region           |start_|end_|
+--------------------------------------+-----------------+------+----+
|Auckland Times                        |Auckland         |1842  |1846|
|Press                                 |Canterbury       |1861  |1945|
|Temuka Leader                         |Canterbury       |1878  |1932|
|Timaru Herald                         |Canterbury       |1864  |1920|
|Rangitikei Advocate and Manawatu Argus|Manawatu-Wanganui|1907  |1920|
|Wanganui Herald                       |Manawatu-Wanganui|1867  |1920|
|Woodville Examiner                    |Manawatu-Wanganui|1883  |1920|
|Evening Star                          |Otago            |1865  |1947|
|Lake Wakatip Mail                     |Otago            |1863  |1947|
|Mt Benger Mail                        |Otago            |1881  |1941|
|Otago Daily Times                     |Otago   

**Notice that in this publisher-region relationship dataframe, there are two publisher's name is not identical with the dataset: "Bay Of Plenty Times" mismatch by "of", "New Zealand Free Lance" mismatch by "New Zeland", so we modify the** `df_region` **to make it identical with dataset:**

In [24]:
# update df_region for Bay Of Plenty Times and New Zealand Free Lance
df_region = df_region.withColumn(
    'publisher_',
    F.when(
        F.col('publisher_') == 'Bay of Plenty Times',
        F.lit('Bay Of Plenty Times')
    ).otherwise(
        F.col('publisher_')
    )
).withColumn(
    'publisher_',
    F.when(
        F.col('publisher_') == 'Free Lance',
        F.lit('New Zealand Free Lance')
    ).otherwise(
        F.col('publisher_')
    )
)

**Check if the two publishers' name were modified:**

In [25]:
df_region.filter((df_region.publisher_ == 'Bay Of Plenty Times') | (df_region.publisher_ == 'New Zealand Free Lance')).show(10, False)

+----------------------+-------------+------+----+
|publisher_            |region       |start_|end_|
+----------------------+-------------+------+----+
|Bay Of Plenty Times   |Bay of Plenty|1872  |1949|
|New Zealand Free Lance|Wellington   |1900  |1920|
+----------------------+-------------+------+----+



**Extract region column, and abandon redundant columns:**

In [26]:
df = (df.join(df_region, df.publisher == df_region.publisher_, how='left')
      .select(F.col('id'), 
              F.col('publisher'), 
              F.col('region'), 
              F.col('date'), 
              F.col('ads'), 
              F.col('title'), 
              F.col('content'))
      )

**Imputing missing value in region column with "unknwon":**

In [27]:
df = df.na.fill({'region':'unknown'})

**Check if miss any field or element:**

In [28]:
print('Print Null and empty string:')
df.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ''), c)).alias(c) for c in df.columns]).show()

Print Null and empty string:
+---+---------+------+----+---+-----+-------+
| id|publisher|region|date|ads|title|content|
+---+---------+------+----+---+-----+-------+
|  0|        0|     0|   0|  0|    0|      0|
+---+---------+------+----+---+-----+-------+



**Check dataframe szie:**

In [29]:
nrow = df.count()
print('Shape of dataframe: ({}, {})'.format(nrow, len(df.columns)))
print('usable line percentage:', nrow/nrow_raw)
print('removed line number:', nrow_raw - nrow)   

Shape of dataframe: (16131646, 7)
usable line percentage: 0.9641437287026962
removed line number: 599932


**After data wrangling, there are:**
* 16,131,646 (96.4%) samples/rows/lines/documents usable, 
* 599,932 samples/rows/lines/documents were removed.

## 6 Save Dataset

**This dataframe would be our final dataset to deal with, let's save it as compressed csv file to save time for later processes.**

In [30]:
path = r'../data/dataset'
df = df.orderBy('id')
df.write.csv(path, mode='overwrite', compression='gzip')

**Check the clean dataset size:**

In [31]:
path = r'../data/papers_past'
print('raw   dataset size:', subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8'))
path = r'../data/dataset'
print('clean dataset size:', subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8'))

raw   dataset size: 33G
clean dataset size: 14G


**After processing and compressing, the dataset reduce from 33GB to 14GB.**

---